# 第1回の宿題

## GPT-4o-mini APIを活用した文字認識されていないPDF文書からのテキスト抽出ツールのStreamlitによるUI

動機としてはNotebookLMでマインドマップ化したいPDFがあったのですが、文字認識されておらずadobeのOCRを試したがその精度も完璧ではなかったためです。

背景としては自身がアクチュアリーという資格の2次試験を今年受験予定であり、その教科書がPDFで公開されているものの文字認識されていないものが数多く含まれていたことです。

文字認識されていないPDFの例:

https://www.actuaries.jp/examin/textbook/pdf/hoken1-seiho_06.pdf

## 環境変数の設定

In [ ]:
!git clone https://github.com/aimitmk/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 52 (delta 9), reused 8 (delta 6), pack-reused 13 (from 1)
Receiving objects: 100% (52/52), 57.96 KiB | 1.93 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

%cd /content/lecture-ai-engineering/day1
load_dotenv(find_dotenv())

/content/lecture-ai-engineering/day1


True

## ライブラリのインストール

In [4]:
%cd /content/lecture-ai-engineering

/content/lecture-ai-engineering


In [5]:
%%capture
!pip install -r requirements.txt

In [6]:
!apt-get update
!apt-get install -y poppler-utils

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,864 kB]
Get:13 https://r2u.stat.illinois.edu/ubu

## ライブラリのインポート

In [7]:
import os
import io
import base64
import tempfile
import time
import PyPDF2
import openai
from tqdm.notebook import tqdm
from pdf2image import convert_from_path, convert_from_bytes
from PIL import Image
import re
import json
from IPython.display import display, HTML

## Streamlitによる対話型UI

以下のセルを実行すると、Streamlitアプリケーションが起動し、ngrokを使用して公開URLが生成されます。
このURLにアクセスすることで、Webブラウザから対話的にPDF文書からのテキスト抽出が可能になります。

In [8]:
!ngrok authtoken $$NGROK_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [9]:
# Streamlitアプリの起動
from pyngrok import ngrok

# 現在のディレクトリを確認
!pwd

# 8501ポートをngrokで公開
public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
print("このURLにアクセスして、Streamlitアプリケーションを使用できます。")
print("注意: このセルを実行している間はアプリケーションが利用可能です。中断するとアクセスできなくなります。")

# Streamlitアプリを実行
!streamlit run pdf_extraction_app.py

/content/lecture-ai-engineering


公開URL: https://95ba-34-132-100-108.ngrok-free.app
このURLにアクセスして、Streamlitアプリケーションを使用できます。
注意: このセルを実行している間はアプリケーションが利用可能です。中断するとアクセスできなくなります。



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.132.100.108:8501

  Stopping...
  Stopping...


In [10]:
# 後片付け: ngrokのトンネルを削除
# このセルは、アプリの使用が終わったら実行してください
from pyngrok import ngrok
ngrok.kill()
print("ngrokのトンネルが削除されました。")

ngrokのトンネルが削除されました。
